### Setup your environnement variables

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR API KEY"
os.environ["UIFORM_API_KEY"] = "YOUR API KEY" # go to https://uiform.com to create your API Key

In [1]:
from dotenv import load_dotenv
import os

assert load_dotenv("../.env.local") # Load environment variables from a .env file
print("UIFORM API KEY:",os.getenv("UIFORM_API_KEY"))

UIFORM API KEY: sk_3ovI3g3Hq7pmBvySp3tP2OCMlZtHSNsAXfKG6VQjiDI


## Load the json schema

In [2]:
from uiform import UiForm, Schema
from pydantic import BaseModel, Field, ConfigDict
import os

uiclient = UiForm()
doc_msg = uiclient.documents.create_messages(
    document = "freight/booking_confirmation.jpg"
)

class CalendarEvent(BaseModel):
    model_config = ConfigDict(json_schema_extra = {"X-SystemPrompt": "You are a useful assistant."})

    name: str = Field(...,
        description="The name of the calendar event.",
        json_schema_extra={"X-FieldPrompt": "Provide a descriptive and concise name for the event."}
    )
    date: str = Field(...,
        description="The date of the calendar event in ISO 8601 format.",
        json_schema_extra={
            'X-ReasoningPrompt': 'The user can mention it in any format, like **next week** or **tomorrow**. Infer the right date format from the user input.',
        }
    )

schema_obj =Schema(
    pydantic_model = CalendarEvent
)


## OpenAI

In [9]:
from openai import OpenAI

client = OpenAI()

#----

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=doc_msg.openai_messages + [
        {
            "role": "user",
            "content": "Summarize the document"
        }
    ]
)

# --- OR 

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": schema_obj.schema_version,
            "schema": schema_obj.inference_json_schema,
            "strict": True
        }
    }
)

# --- OR 

completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format=schema_obj.inference_pydantic_model
)



# xAI

In [11]:
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("XAI_API_KEY"),
    base_url="https://api.x.ai/v1"
)

#----

completion = client.chat.completions.create(
    model="grok-2-vision-1212",
    messages=doc_msg.openai_messages + [
        {
            "role": "user",
            "content": "Summarize the document"
        }
    ]
)

# --- OR 

completion = client.chat.completions.create(
    model="grok-2-vision-1212",
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": schema_obj.schema_version,
            "schema": schema_obj.inference_json_schema,
            "strict": True
        }
    }
)

# --- OR 

completion = client.beta.chat.completions.parse(
    model="grok-2-vision-1212",
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format=schema_obj.inference_pydantic_model
)




# Gemini

In [12]:
from openai import OpenAI

client = OpenAI(
    api_key = os.getenv("GEMINI_API_KEY"),
    base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
)

#----

completion = client.chat.completions.create(
    model="gemini-1.5-flash",
    messages=doc_msg.openai_messages + [
        {
            "role": "user",
            "content": "Summarize the document"
        }
    ]
)


# --- OR 

completion = client.chat.completions.create(
    model="gemini-1.5-flash",
    messages=schema_obj.openai_messages + doc_msg.openai_messages + [
        {
            "role": "user",
            "content": "Extract informations from this document."
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": schema_obj.schema_version,
            "schema": schema_obj.inference_gemini_json_schema,
            "strict": True
        }
    }
)

# --- Does not work with beta.chat.completions.parse !

# Anthropic

In [9]:
from anthropic import Anthropic

client = Anthropic(api_key=os.getenv("CLAUDE_API_KEY"))

#----

completion = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1000,
    temperature=0,
    system=doc_msg.anthropic_system_prompt,
    messages=doc_msg.anthropic_messages + [
        {
            "role": "user",
            "content": "Summarize this Document"
        }
    ]
)

# --- OR 

# Using Anthropic's JSON mode
completion = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1000,
    temperature=0,
    system=schema_obj.anthropic_system_prompt + doc_msg.anthropic_system_prompt,
    messages=schema_obj.anthropic_messages + doc_msg.anthropic_messages
)

from anthropic.types.text_block import TextBlock
assert isinstance(completion.content[0], TextBlock)
print(completion.content[0].text)


{
  "name": "Dangerous Goods Delivery - Munich",
  "reasoning___date": "The delivery is scheduled for 5/3/2023 from 10:00 to 16:00",
  "date": "2023-05-03T10:00:00Z"
}
